In [ ]:
name = "My full name"  # write YOUR NAME

honorPledge = "I affirm that I have not given or received any unauthorized help on this assignment, \
               and that this work is my own.\n"


print("\nName: ", name)
print("\nHonor pledge: ", honorPledge)

# MAC0460/MAC5832 (2021)


## EP1 -- The perceptron algorithm

The goal of this EP is to implement the perceptron algorithm. For simplicity, we will implement for and test it on datapoints in $\mathbb{R}^2$. 

The datapoints are assumed to be stored in a numpy array.
This is also an opportunity to start getting used to Python and Numpy, for those that are not used yet.

<hr>

Let $X = \Big\{ (\mathbf{x}^{(1)},y^{(1)}), (\mathbf{x}^{(2)},y^{(2)}), \ldots, (\mathbf{x}^{(N)},y^{(N)})\Big\}$, $\mathbf{x}^{(i)} \in \mathbb{R}^d$ and $y^{(i)} \in \{-1, +1\}$, $i=1,2,\ldots,N$, be a set of training examples. We will assume they are linearly separable (meaning that there is a hyperplane that separates positive from negative examples).

Given $\mathbf{x} = (x_1, \ldots, x_d)  \in \mathbb{R}^d$, let $\tilde{\mathbf{x}} = (1, x_1, \ldots, x_d) \in \mathbb{R}^{1+d}$.

The perceptron algorithm determines a weight vector $\mathbf{w} \in \mathbb{R}^{1+d}$ such that $\mathrm{sign}(\mathbf{w}^T \tilde{\mathbf{x}}^{(i)}) = y^{(i)}$, $\forall i$, where

$$
sign(z) = \left\{\begin{array}{ll}+1, & \mbox{if $z \geq 0$,}\\ -1, & \mbox{if $z < 0$.}\\
\end{array}\right.
$$




## Creating a training set 

- Our dataset will consist of $N$ points. Let us choose some points in the $[-1,2]\times [-1,2] \subseteq \mathbb{R}^2$ region.
- Let us fix a function $f(x_1,x_2) = w_0 + w_1\,x_1 + w_2\,x_2$, such that its zero points form a line that partitions the $[-1,2]\times [-1,2]$ square into two regions; consider $w_2 \neq 0$
- We define the points $(x_1,x_2)$ such that $f(x_1,x_2) \geq 0$ as the positive examples and those such that $f(x_1,x_2) < 0$ as the negative ones (that is, the class label of a point is determined by the signal of $f$ at that point).
- Let us also plot a graph of the examples (<font color="blue">positives</font> in blue and   <font color="red">negatives</font> in red) and also of the decision surface.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Create a numpy array with N points
xmin = ymin = -1
xmax =ymax = 2

X = np.asarray([[1.3, -0.2],[0,0],[0,1],[1,0],[1,1]])
print("Shape of array X: ", X.shape)
N = X.shape[0]
print("Number of examples: ", N)

# add a left column with 1's into X -- X extended,
# that way Xi has the same number of elements of the weight array
def add_column_of_ones(X):
    return np.hstack(( np.ones((X.shape[0],1)), X ) )
Xe = add_column_of_ones(X)
print("Shape of array Xe: ", Xe.shape)

# define a target weight vector
w_target = np.asarray([[0.5],[-1], [1]])
print("Shape of array w_target: ", w_target.shape)
print("Target weight array: \n", w_target)

# define y (class) values, based on the line defined by the target weight vector
y = np.sign(np.dot(Xe, w_target))
print("Shape of array y: ", y.shape)


# Plotting ...

# plot the line
a = -w_target[1] / w_target[2] # slope  -- we will have trouble if w_target[2]=0 ...
b = -w_target[0] / w_target[2] # intercept
x_l = np.linspace(xmin, xmax, 50)
y_l = a*x_l + b

plt.figure(figsize=(6,6))
plt.plot(x_l, y_l);
axes = plt.gca()
axes.set_xlim([xmin,xmax])
axes.set_ylim([xmin,ymax])

# Determine the colors of each of the examples
colors = ["blue" if y[i]==1 else "red" for i in range(N)]
print("color: ", colors)

# plot the examples
plt.scatter(X[:,0],X[:,1],c=colors)



## Weight initialization

Let us start with a weight vector that clearly is not too close to the target weights. This will be our initial hypothesis.

To see how our hypothesis is classifying the examples, let us use the following convention.

We will use the colors (red or blue) to indicate the target label (ground-truth): <font color="blue">positive</font> and <font color="red">negative</font>

To indicate how the current hypothesis is classifying the examples we will use markers as follows:
- o : corretly classified by the hypothesis<br>
- x : wrongly classified by the hypothesis<br>


In [ ]:

def plot_state(Xe,w,xmin=-1,xmax=2,ymin=-1,ymax=2):
    # compute yhat - prediction
    yhat = np.sign(np.dot(Xe,w))

    correct = np.where(y == yhat)[0]
    misclassified = np.where(y != yhat)[0]

    colors_o = ["blue" if y[i]==1 else "red" for i in correct]
    colors_x = ["blue" if y[i]==1 else "red" for i in misclassified]

    # plotting
    a = -w[1] / w[2] # slope
    b = -w[0] / w[2] # intercept
    x_l = np.linspace(-1, 2, 50)
    y_l = a*x_l + b

    plt.figure(figsize=(6,6))
    plt.plot(x_l, y_l);
    axes = plt.gca()
    axes.set_xlim([xmin,xmax])
    axes.set_ylim([xmin,ymax])
    plt.title("Weight:   w0=%.2f    w1=%.2f    w2=%.2f" %(w[0],w[1],w[2]))
    plt.scatter(X[correct,0],X[correct,1],c=colors_o, marker='o');
    plt.scatter(X[misclassified,0],X[misclassified,1],c=colors_x, marker='x');

    
# Starting weight vector <---- change as you wish (as long as w0[2] != 0)
w0 = np.asarray([[-0.5], [1] , [1]])
   
plot_state(Xe,w0)


# Exercise

Implement the PERCEPTRON algorithm, following the specification in the next box.

Your algorithm should process the examples in a round-robin fashion (i.e., cyclically, from 0 to $N-1$, and then back to 0 and so on).

NOTE, however, that you should avoid loops (for, while) that scan arrays, index by index. Use Numpy methods.

If parameter <tt>plot=True</tt>, the "state" at the beginning of each iteration must be plotted


<b>Suggestion:</b> Rather than plotting the state at each iteration, you can write a code that generates an animation that shows the sequence of the states.


In [ ]:
def perceptron(Xe,y,w0,plot=False):
    """
    Parameters:
       Xe   : ndarray (N,d+1) - it already has the 1's in column 0
       y    : ndarray (N,1)
       w0   : ndarray (d+1,1) - the initial weight vector
       plot : If True, plot the state at the beginning of each iteration
       
    Returns:
       w : ndarray (d+1,) - the final weight vector
    """
    
    # START OF YOUR CODE:
    raise NotImplementedError("Function perceptron() is not implemented")
    # END YOUR CODE
    
    return w


# Test your function for w0, X and y as defined above
w0 = np.asarray([[-0.5], [1] , [1]]) # <---- you can change it (as long as w0[2] != 0)
print("Initial weight vector=\n", w0)

w = perceptron(Xe,y,w0,plot=True)


# Print the final weight vector and plot the final graph using the function plot_state.
# START OF YOUR CODE:
# END YOUR CODE


# What else could be explored

Feel free to experiment some modifications in your code (just add new code cells, if that is the case):

0. Change the set of examples $X$ or the initial weight vector and run the code
1. You could change the code so that the initial weight vector is initialized with ramdom numbers
2. You could also generate the set of examples $\mathbf{x}$ randomly (for instance, points randomly selected from two Gaussian distributions)
3. What do you need to change in order to have your code ready for $d>2$ dimensional data ?
4. How do you need to change your algorithm in order to have the POCKET version, so that it will work on non-linearly separable data?